In [1]:
import numpy as np
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from augmentation import *
from model_resgcn import ResGCN
from data import ProcessDataset

%load_ext autoreload
%autoreload 2

In [2]:
file_name = "casia-b_pose_train.csv"  
csv_path = os.path.join("..", "data", file_name)

In [44]:
df_test = pd.read_csv(csv_path)
print(df_test.shape)

(609075, 52)


In [ ]:
# Tạo dataset không dùng transform để test trước
dataset = ProcessDataset(csv_path=csv_path)
print("Dataset size:", len(dataset))

Dataset size: 6490


In [46]:
for i in range(len(dataset)):
    sample = dataset[i]
    node_features = sample["node_features"]
    print(f"Sample {i} - Shape: {node_features.shape}")

Sample 0 - Shape: torch.Size([60, 17, 3])
Sample 1 - Shape: torch.Size([60, 17, 3])
Sample 2 - Shape: torch.Size([60, 17, 3])
Sample 3 - Shape: torch.Size([60, 17, 3])
Sample 4 - Shape: torch.Size([60, 17, 3])
Sample 5 - Shape: torch.Size([60, 17, 3])
Sample 6 - Shape: torch.Size([60, 17, 3])
Sample 7 - Shape: torch.Size([60, 17, 3])
Sample 8 - Shape: torch.Size([60, 17, 3])
Sample 9 - Shape: torch.Size([60, 17, 3])
Sample 10 - Shape: torch.Size([60, 17, 3])
Sample 11 - Shape: torch.Size([60, 17, 3])
Sample 12 - Shape: torch.Size([60, 17, 3])
Sample 13 - Shape: torch.Size([60, 17, 3])
Sample 14 - Shape: torch.Size([60, 17, 3])
Sample 15 - Shape: torch.Size([60, 17, 3])
Sample 16 - Shape: torch.Size([60, 17, 3])
Sample 17 - Shape: torch.Size([60, 17, 3])
Sample 18 - Shape: torch.Size([60, 17, 3])
Sample 19 - Shape: torch.Size([60, 17, 3])
Sample 20 - Shape: torch.Size([60, 17, 3])
Sample 21 - Shape: torch.Size([60, 17, 3])
Sample 22 - Shape: torch.Size([60, 17, 3])
Sample 23 - Shape: to

In [47]:
count = 0
for i, sample in enumerate(dataset):
    node_features = sample["node_features"]
    T = node_features.shape[0]
    if T < 60:
        print(f"Sample {i}:")
        print(f"  subject_id       : {sample['subject_id']}")
        print(f"  condition        : {sample['condition']}")
        print(f"  condition_index  : {sample['condition_index']}")
        print(f"  angle            : {sample['angle']}")
        print(f"  num_frames (T)   : {T}")
        print("-" * 40)
        count += 1
    if count >= 1:
        break


Sample 128:
  subject_id       : 2
  condition        : bg
  condition_index  : 2
  angle            : 126
  num_frames (T)   : 59
----------------------------------------


In [54]:
sample = dataset[128]
node_features = sample["node_features"]
print(node_features.shape)
print(f"{node_features}")

torch.Size([59, 17, 3])
tensor([[[3.1550e+02, 6.5205e+01, 9.0128e-01],
         [3.1659e+02, 6.0830e+01, 9.5654e-01],
         [3.1659e+02, 6.0830e+01, 9.0047e-01],
         ...,
         [3.1221e+02, 1.6035e+02, 8.4212e-01],
         [2.9034e+02, 1.8769e+02, 7.7959e-01],
         [2.9144e+02, 1.8769e+02, 8.5145e-01]],

        [[3.1067e+02, 6.6837e+01, 9.6956e-01],
         [3.1172e+02, 6.3686e+01, 9.5321e-01],
         [3.1172e+02, 6.3686e+01, 9.4436e-01],
         ...,
         [3.0542e+02, 1.6032e+02, 7.3060e-01],
         [2.8652e+02, 1.8867e+02, 7.3620e-01],
         [2.8652e+02, 1.8867e+02, 8.6745e-01]],

        [[3.0673e+02, 6.7550e+01, 9.5923e-01],
         [3.0673e+02, 6.4532e+01, 9.5325e-01],
         [3.0673e+02, 6.4532e+01, 9.0954e-01],
         ...,
         [3.0070e+02, 1.6011e+02, 8.8205e-01],
         [3.0372e+02, 1.8929e+02, 8.5408e-02],
         [2.8359e+02, 1.9130e+02, 8.7106e-01]],

        ...,

        [[1.0831e+02, 6.6113e+01, 9.4681e-01],
         [1.0831e+02,

In [ ]:
# Ví dụ tạo transform
transforms2 = TransformCompose([
    InterpolateFrames(probability=1.0, target_frames=60)
    # FlipSequence(probability=0.3),
    # ShuffleSequence(enabled=True)
])

# Dùng lại dataset với transform
dataset2 = ProcessDataset(csv_path=csv_path, transform=transforms2)

print("Dataset size:", len(dataset2))


Dataset size: 6490


In [50]:
for i in range(len(dataset2)):
    sample2 = dataset2[i]
    node_features2 = sample2["node_features"]
    print(f"Sample {i} - Shape: {node_features2.shape}")

Sample 0 - Shape: torch.Size([60, 17, 3])
Sample 1 - Shape: torch.Size([60, 17, 3])
Sample 2 - Shape: torch.Size([60, 17, 3])
Sample 3 - Shape: torch.Size([60, 17, 3])
Sample 4 - Shape: torch.Size([60, 17, 3])
Sample 5 - Shape: torch.Size([60, 17, 3])
Sample 6 - Shape: torch.Size([60, 17, 3])
Sample 7 - Shape: torch.Size([60, 17, 3])
Sample 8 - Shape: torch.Size([60, 17, 3])
Sample 9 - Shape: torch.Size([60, 17, 3])
Sample 10 - Shape: torch.Size([60, 17, 3])
Sample 11 - Shape: torch.Size([60, 17, 3])
Sample 12 - Shape: torch.Size([60, 17, 3])
Sample 13 - Shape: torch.Size([60, 17, 3])
Sample 14 - Shape: torch.Size([60, 17, 3])
Sample 15 - Shape: torch.Size([60, 17, 3])
Sample 16 - Shape: torch.Size([60, 17, 3])
Sample 17 - Shape: torch.Size([60, 17, 3])
Sample 18 - Shape: torch.Size([60, 17, 3])
Sample 19 - Shape: torch.Size([60, 17, 3])
Sample 20 - Shape: torch.Size([60, 17, 3])
Sample 21 - Shape: torch.Size([60, 17, 3])
Sample 22 - Shape: torch.Size([60, 17, 3])
Sample 23 - Shape: to

In [53]:
sample2 = dataset2[128]
node_features2 = sample2["node_features"]
print(node_features2.shape)
print(f"{node_features2}")

torch.Size([60, 17, 3])
tensor([[[3.1550e+02, 6.5205e+01, 9.0128e-01],
         [3.1659e+02, 6.0830e+01, 9.5654e-01],
         [3.1659e+02, 6.0830e+01, 9.0047e-01],
         ...,
         [3.1221e+02, 1.6035e+02, 8.4212e-01],
         [2.9034e+02, 1.8769e+02, 7.7959e-01],
         [2.9144e+02, 1.8769e+02, 8.5145e-01]],

        [[3.1067e+02, 6.6837e+01, 9.6956e-01],
         [3.1172e+02, 6.3686e+01, 9.5321e-01],
         [3.1172e+02, 6.3686e+01, 9.4436e-01],
         ...,
         [3.0542e+02, 1.6032e+02, 7.3060e-01],
         [2.8652e+02, 1.8867e+02, 7.3620e-01],
         [2.8652e+02, 1.8867e+02, 8.6745e-01]],

        [[3.0673e+02, 6.7550e+01, 9.5923e-01],
         [3.0673e+02, 6.4532e+01, 9.5325e-01],
         [3.0673e+02, 6.4532e+01, 9.0954e-01],
         ...,
         [3.0070e+02, 1.6011e+02, 8.8205e-01],
         [3.0372e+02, 1.8929e+02, 8.5408e-02],
         [2.8359e+02, 1.9130e+02, 8.7106e-01]],

        ...,

        [[1.0349e+02, 6.7027e+01, 9.3941e-01],
         [1.0420e+02,

In [3]:
# Ví dụ tạo transform
transforms3 = TransformCompose([
    InterpolateFrames(),
    FlipSequence(probability=1.0)
    # ShuffleSequence(enabled=True)
])

# Dùng lại dataset với transform
dataset3 = ProcessDataset(csv_path=csv_path, transform=transforms3)

print("Dataset size:", len(dataset3))


Dataset size: 6490


In [4]:
sample3 = dataset3[0]
node_features3 = sample3["node_features"]
print(node_features3.shape)
print(f"{node_features3}")

torch.Size([60, 17, 3])
tensor([[[148.4712,  45.8011,   0.9490],
         [151.1370,  41.8024,   0.9682],
         [143.1396,  43.1353,   0.9550],
         ...,
         [143.1396, 173.7603,   0.9006],
         [159.1345, 200.4185,   0.8911],
         [144.4725, 223.0780,   0.8431]],

        [[148.5247,  44.7782,   0.9699],
         [152.4901,  40.8128,   0.9666],
         [144.5593,  40.8128,   0.9590],
         ...,
         [143.2375, 171.6714,   0.8846],
         [159.0992, 200.7511,   0.8536],
         [145.8811, 219.2564,   0.8616]],

        [[149.7049,  44.2601,   0.9774],
         [153.5435,  40.4215,   0.9702],
         [145.8663,  40.4215,   0.9655],
         ...,
         [144.5868, 169.6546,   0.9069],
         [159.9412, 202.9226,   0.8772],
         [145.8663, 211.8793,   0.8914]],

        ...,

        [[151.2094,  54.2774,   0.9455],
         [152.4863,  52.3620,   0.9612],
         [149.2940,  52.3620,   0.9499],
         ...,
         [149.9325, 114.9310,   0.8580]